In [ ]:
!git clone https://github.com/aashishkhimasia/google-research

Cloning into 'google-research'...
remote: Enumerating objects: 58111, done.
remote: Counting objects: 100% (523/523), done.
remote: Compressing objects: 100% (384/384), done.
remote: Total 58111 (delta 248), reused 368 (delta 130), pack-reused 57588
Receiving objects: 100% (58111/58111), 361.33 MiB | 20.27 MiB/s, done.
Resolving deltas: 100% (34066/34066), done.
Checking out files: 100% (16574/16574), done.


In [ ]:
%pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 489.6 MB 15 kB/s 
     |████████████████████████████████| 1.3 MB 46.3 MB/s 
     |████████████████████████████████| 463 kB 67.1 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.9.2
    Uninstalling tensorflow-2.9.2:
      Successfully uninstalled tensorflow-2.9.2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 4.8 MB/s 
Looking in indexes: https://pypi.org/simple, ht

In [ ]:
import tensorflow as tf

In [ ]:
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
print('Running on TPU ', tpu.cluster_spec().as_dict())

Running on TPU  {'worker': ['10.43.36.114:8470']}


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/google-research

/content/google-research


In [ ]:
!tar -xzf "gs://dissertation-adk/pretrained_emb_voc/vision_vatt_misc_data.tgz" -C "/content/google-research/vatt/misc/"     #[run this cell to extract tar.gz files]

tar (child): Cannot connect to gs: resolve failed

gzip: stdin: unexpected end of file
tar: Child returned status 128
tar: Error is not recoverable: exiting now


In [ ]:
# get the file in, see if that helps with the howto100m

In [ ]:
!tar -xvzf "gs://dissertation-adk/pretrained_emb_voc/vision_vatt_misc_data.tgz" --force-local


tar (child): gs\://dissertation-adk/pretrained_emb_voc/vision_vatt_misc_data.tgz: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now


Main.py from vatt (with edits from LG)

In [ ]:
import pprint

# from absl import app
# from absl import flags
from absl import logging

from vatt.configs import factory as config_factory
from vatt.experiments import finetune
from vatt.experiments import pretrain

# flags.DEFINE_string('task', 'PRETRAIN', 'PRETRAIN or FINETUNE.')
# flags.DEFINE_string('mode', 'train', 'train or eval.')
# flags.DEFINE_string('model_dir', None, 'Default path for the experiment.')
# flags.DEFINE_string('model_arch', 'Tx_FAC', 'Arch of the model.')
# flags.DEFINE_string('override_checkpoint', None,
#                     ('Path to a checkpoint for initialization. '
#                      'If this is passed, the model is initialized from this '
#                      'checkpoint, even if there is a valid latest checkpoint '
#                      'inside the model_dir.'))
# flags.DEFINE_string('config_file', None,
#                     ('Path to a YAML config file containing the dictionary of '
#                      'parameters to override the original params defined '
#                      'under configs/'))
# flags.DEFINE_string('params_override', None,
#                     'A safe_dumped str of a dictionary of parameters')
# flags.DEFINE_string('strategy_type', 'tpu', 'Type of the distribution strategy')
# flags.DEFINE_string('tpu', None, 'Address of the TPU device')


# FLAGS = flags.FLAGS


def get_params():
  """Constructs the configuration of the experiment."""

  task = "FINETUNE"  # FLAGS.task
  model_arch = "Tx_FAC"  # FLAGS.model_arch
  params = config_factory.build_experiment_configs(
      task=task,
      model_arch=model_arch,
      )

  # if FLAGS.config_file:
  #   params.override_from_file(FLAGS.config_file)

 # if FLAGS.params_override:
 #    params.override_from_str(FLAGS.params_override)

  params.override({
      'mode': "eval",  # FLAGS.mode,
      'model_dir': None, # FLAGS.model_dir,
      'checkpoint_path': 'gs://dissertation-adk/checkpoint', #None, # FLAGS.override_checkpoint,
      'strategy_config': {'tpu': None, # FLAGS.tpu,
                          'distribution_strategy': "tpu" # FLAGS.strategy_type
                          },
  })

  return params



def main():
  # del argv  # Unused.

  params = get_params()
  logging.info('Model Parameters: %s', pprint.pformat(params.as_dict()))

  print(str(params.model_dir))
  params.eval.input.name = "toy_ds"
  if params.task.lower() == 'pretrain':
    executor = pretrain.get_executor(params=params)

  elif params.task.lower() == 'finetune':
    executor = finetune.get_executor(params=params)

  else:
    raise ValueError('Task not found: %s.' % params.task)

  return executor.run(mode=params.mode)


if __name__ == '__main__':
  # app.run(main)
  main()

None


NotFoundError: ignored

In [ ]:
%ls

aav/
abps/
abstract_nas/
action_angle_networks/
action_gap_rl/
activation_clustering/
adaptive_learning_rate_tuner/
adaptive_prediction/
adaptive_surrogates/
after_kernel/
albert/
al_for_fep/
algae_dice/
Algorithms_and_Hardness_for_Learning_Linear_Thresholds_from_Label_Proportions/
aloe/
alx/
amortized_bo/
anthea/
aptamers_mlpd/
aqt/
aquadem/
arxiv_latex_cleaner/
assemblenet/
assessment_plan_modeling/
attentional_adapters/
attribution/
automatic_structured_vi/
automl_zero/
autoregressive_diffusion/
aux_tasks/
axial/
bam/
bangbang_qaoa/
basisnet/
batch_science/
behavior_regularized_offline_rl/
bertseq2seq/
better_storylines/
bigg/
bisimulation_aaai2020/
bitempered_loss/
blur/
bnn_hmc/
bonus_based_exploration/
building_detection/
bustle/
cache_replacement/
caltrain/
capsule_em/
caql/
cascaded_networks/
cate/
cell_embedder/
cell_mixer/
cfq/
cfq_pt_vs_sa/
charformer/
CIQA/
ciw_label_noise/
class_balanced_distillation/
clay/
c_learning/
cluster_gcn/
clustering_normalized_cuts/
cnn_quantizat